In [1]:
#add confident interval
import config_cat_embedding
import pandas as pd
import numpy as np
import random
import time
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
# from sklearn.svm import SVC
from xgboost import XGBClassifier
from scikeras.wrappers import KerasClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score)
from scipy import stats

from data_prep import bank_data_prep, adult_data_prep
from embedding_helper import create_network

# Set the random seed for reproducibility
random.seed(42)
np.random.seed(42)

# Load and preprocess data
data_path = config_cat_embedding.paths['data']
bank_data = pd.read_csv(data_path+'adult.csv', sep=',')
df_bank, cat_cols = adult_data_prep(bank_data)

X = df_bank.iloc[:, :-1]
y = df_bank.y

# Define the classifiers
seed = 42
# We will determine the number_of_features inside the cross-validation loop after preprocessing

models = [
    ('LR', LogisticRegression(solver='lbfgs', random_state=seed, max_iter=1000)),
    ('DT', DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=seed)),
    ('RF', RandomForestClassifier(n_estimators=200, max_depth=5, random_state=seed, min_samples_leaf=3)),
    ('KNN', KNeighborsClassifier(n_neighbors=3)),
    ('XGB', XGBClassifier(eval_metric='logloss', random_state=seed)),
    # ('SVM', SVC(gamma='scale', random_state=seed, probability=True)),
    ('MLP', KerasClassifier(
        model=create_network,
        epochs=100, batch_size=100, verbose=0, random_state=seed))
]

# Cross-validation setup
cv = StratifiedKFold(n_splits=20, shuffle=True, random_state=seed)

# Function to calculate confidence intervals
def confidence_interval(data, confidence=0.95):
    n = len(data)
    m = np.mean(data)
    std_err = stats.sem(data)
    h = std_err * stats.t.ppf((1 + confidence) / 2, n - 1)
    return m, m - h, m + h

# Loop over models
for name, model in models:
    print(f"Classifier: {name}")
    # Lists to store metrics for each fold
    accuracies = []
    precisions = []
    recalls = []
    f1s = []
    roc_aucs = []
    
    # Start the timer before cross-validation
    start_time = time.time()
    
    fold = 1
    for train_index, test_index in cv.split(X, y):
        # Split the data into training and testing sets for this fold
        X_train_fold, X_test_fold = X.iloc[train_index], X.iloc[test_index]
        y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]
        
        # Preprocess data within the fold
        # One-hot encoding
        import category_encoders as ce
        one_hot_encoder = ce.OrdinalEncoder(cols=cat_cols) 
        X_train_ohe = one_hot_encoder.fit_transform(X_train_fold)
        X_test_ohe = one_hot_encoder.transform(X_test_fold)
        
        # Standard scaling
        stc = StandardScaler()
        X_train_scaled = stc.fit_transform(X_train_ohe)
        X_test_scaled = stc.transform(X_test_ohe)
        
        # Update number_of_features for MLP
        number_of_features = X_train_scaled.shape[1]
        if name == 'MLP':
            # Update the model with the correct number of features
            model.set_params(model__number_of_features=number_of_features)
        
        # Fit the model
        model.fit(X_train_scaled, y_train_fold)
        # Predict on the test fold
        y_pred_fold = model.predict(X_test_scaled)
        
        # Get prediction probabilities for ROC AUC
        if hasattr(model, "predict_proba"):
            y_pred_prob_fold = model.predict_proba(X_test_scaled)[:, 1]
        else:
            # For classifiers without predict_proba, use decision_function
            y_pred_prob_fold = model.decision_function(X_test_scaled)
            # Normalize the decision function output to [0,1] range
            y_pred_prob_fold = (y_pred_prob_fold - y_pred_prob_fold.min()) / (y_pred_prob_fold.max() - y_pred_prob_fold.min())
        
        # Calculate metrics
        accuracies.append(accuracy_score(y_test_fold, y_pred_fold))
        precisions.append(precision_score(y_test_fold, y_pred_fold, zero_division=0))
        recalls.append(recall_score(y_test_fold, y_pred_fold))
        f1s.append(f1_score(y_test_fold, y_pred_fold))
        roc_aucs.append(roc_auc_score(y_test_fold, y_pred_prob_fold))
        
       # print(f"Fold {fold} completed.")
        fold += 1
    
    # Stop the timer after cross-validation
    end_time = time.time()
    total_computation_time = end_time - start_time  # Total time for the model
    
    # Calculate mean and confidence intervals
    acc_mean, acc_ci_lower, acc_ci_upper = confidence_interval(accuracies)
    prec_mean, prec_ci_lower, prec_ci_upper = confidence_interval(precisions)
    rec_mean, rec_ci_lower, rec_ci_upper = confidence_interval(recalls)
    f1_mean, f1_ci_lower, f1_ci_upper = confidence_interval(f1s)
    roc_mean, roc_ci_lower, roc_ci_upper = confidence_interval(roc_aucs)
    
    # Print results
    print(f"Accuracy: {acc_mean:.3f} (95% CI: {acc_ci_lower:.3f} - {acc_ci_upper:.3f})")
    print(f"Precision: {prec_mean:.3f} (95% CI: {prec_ci_lower:.3f} - {prec_ci_upper:.3f})")
    print(f"Recall: {rec_mean:.3f} (95% CI: {rec_ci_lower:.3f} - {rec_ci_upper:.3f})")
    print(f"F1 Score: {f1_mean:.3f} (95% CI: {f1_ci_lower:.3f} - {f1_ci_upper:.3f})")
    print(f"ROC AUC: {roc_mean:.3f} (95% CI: {roc_ci_lower:.3f} - {roc_ci_upper:.3f})")
    print(f"Total Computation Time: {total_computation_time:.3f} seconds\n")


Classifier: LR
Accuracy: 0.797 (95% CI: 0.794 - 0.800)
Precision: 0.684 (95% CI: 0.672 - 0.695)
Recall: 0.339 (95% CI: 0.329 - 0.350)
F1 Score: 0.453 (95% CI: 0.443 - 0.464)
ROC AUC: 0.813 (95% CI: 0.809 - 0.817)
Total Computation Time: 4.216 seconds

Classifier: DT
Accuracy: 0.799 (95% CI: 0.796 - 0.801)
Precision: 0.984 (95% CI: 0.977 - 0.991)
Recall: 0.190 (95% CI: 0.178 - 0.203)
F1 Score: 0.318 (95% CI: 0.302 - 0.335)
ROC AUC: 0.820 (95% CI: 0.816 - 0.823)
Total Computation Time: 4.711 seconds

Classifier: RF
Accuracy: 0.832 (95% CI: 0.829 - 0.835)
Precision: 0.862 (95% CI: 0.851 - 0.873)
Recall: 0.384 (95% CI: 0.370 - 0.398)
F1 Score: 0.531 (95% CI: 0.517 - 0.544)
ROC AUC: 0.891 (95% CI: 0.887 - 0.894)
Total Computation Time: 74.386 seconds

Classifier: KNN
Accuracy: 0.812 (95% CI: 0.808 - 0.816)
Precision: 0.630 (95% CI: 0.620 - 0.640)
Recall: 0.588 (95% CI: 0.576 - 0.599)
F1 Score: 0.608 (95% CI: 0.599 - 0.617)
ROC AUC: 0.813 (95% CI: 0.806 - 0.820)
Total Computation Time: 72.72

In [22]:
import config_cat_embedding
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
#from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
#from keras.wrappers.scikit_learn import KerasClassifier
from scikeras.wrappers import KerasClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from tqdm import tqdm_notebook as tqdm

from data_prep import bank_data_prep, adult_data_prep
from embedding_helper import create_network
import time
import random

# Set the random seed for reproducibility
random.seed(42)
np.random.seed(42)


data_path = config_cat_embedding.paths['data']
data_path_out = config_cat_embedding.paths['data_output']
bank_data = pd.read_csv(data_path+'adult.csv', sep=',')
df_bank, cat_cols = adult_data_prep(bank_data)

X = df_bank.iloc[:, :-1]
target = df_bank.y

X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.2, random_state=1500)

# One-hot encoding
import category_encoders as ce
label_encoder = ce.OrdinalEncoder(cols=cat_cols) 
X_train_ohe = label_encoder.fit_transform(X_train)
X_test_ohe = label_encoder.transform(X_test)  # Use transform() instead of fit_transform()

# Standard scaling
stc = StandardScaler()
X_train_scaled = stc.fit_transform(X_train_ohe)
X_test_scaled = stc.transform(X_test_ohe)

# Define the classifiers
seed = 42
number_of_features = X_train_scaled.shape[1]  # Number of features in the input data

models = [
    ('LR', LogisticRegression(solver='lbfgs', random_state=seed, max_iter=1000)),
    ('DT', DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=seed)),
    ('RF', RandomForestClassifier(n_estimators=200, max_depth=5, random_state=seed, min_samples_leaf=3)),
    ('KNN', KNeighborsClassifier(n_neighbors=3)),
    ('XGB', XGBClassifier(eval_metric='logloss')),
    ('SVM', SVC(gamma='scale', random_state=seed, probability=True)),
    ('MLP', KerasClassifier(build_fn=create_network, number_of_features=number_of_features, epochs=100, batch_size=100, verbose=0))
]

for name, model in models:
    start_time = time.time()  # Start time
    
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    y_pred_prob = model.predict_proba(X_test_scaled)
    
    end_time = time.time()  # End time
    computation_time = end_time - start_time  # Computation time
    
    print(f"Classifier: {name}")
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred, digits=3))  # Set digits to 3 for three decimal places
    print(f"ROC AUC Score: {roc_auc_score(y_test, y_pred_prob[:, 1]):.3f}")
    print(f"Computation Time: {computation_time:.3f} seconds")
    print()



Classifier: LR
[[6394  424]
 [1405  822]]
              precision    recall  f1-score   support

           0      0.820     0.938     0.875      6818
           1      0.660     0.369     0.473      2227

    accuracy                          0.798      9045
   macro avg      0.740     0.653     0.674      9045
weighted avg      0.780     0.798     0.776      9045

ROC AUC Score: 0.849
Computation Time: 0.171 seconds

Classifier: DT
[[6773   45]
 [1637  590]]
              precision    recall  f1-score   support

           0      0.805     0.993     0.890      6818
           1      0.929     0.265     0.412      2227

    accuracy                          0.814      9045
   macro avg      0.867     0.629     0.651      9045
weighted avg      0.836     0.814     0.772      9045

ROC AUC Score: 0.827
Computation Time: 0.048 seconds

Classifier: RF
[[6647  171]
 [1322  905]]
              precision    recall  f1-score   support

           0      0.834     0.975     0.899      6818
   

C:\Users\P70071469\AppData\Local\anaconda3\envs\SLR\lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\P70071469\AppData\Local\anaconda3\envs\SLR\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Classifier: MLP
[[6259  559]
 [ 940 1287]]
              precision    recall  f1-score   support

           0      0.869     0.918     0.893      6818
           1      0.697     0.578     0.632      2227

    accuracy                          0.834      9045
   macro avg      0.783     0.748     0.763      9045
weighted avg      0.827     0.834     0.829      9045

ROC AUC Score: 0.886
Computation Time: 42.452 seconds



In [23]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.base import clone

# Define number of runs
n_runs = 5
kf = StratifiedKFold(n_splits=n_runs, shuffle=True, random_state=seed)

# Initialize list to store results
results = {}

# Loop over the models
for name, model in models:
    print(f"Classifier: {name}")
    
    roc_auc_scores = []
    computation_times = []
    
    for train_idx, test_idx in kf.split(X_train_scaled, y_train):
        # Clone the model to ensure fresh training
        clf = clone(model)
        
        X_train_fold, X_test_fold = X_train_scaled[train_idx], X_train_scaled[test_idx]
        y_train_fold, y_test_fold = y_train.iloc[train_idx], y_train.iloc[test_idx]
        
        start_time = time.time()  # Start time
        
        clf.fit(X_train_fold, y_train_fold)
        y_pred_prob = clf.predict_proba(X_test_fold)[:, 1]  # Get probabilities
        
        end_time = time.time()  # End time
        computation_time = end_time - start_time  # Computation time
        
        # Calculate ROC AUC score
        roc_auc = roc_auc_score(y_test_fold, y_pred_prob)
        roc_auc_scores.append(roc_auc)
        computation_times.append(computation_time)
    
    # Compute mean and standard deviation of the ROC AUC scores and computation time
    mean_roc_auc = np.mean(roc_auc_scores)
    std_roc_auc = np.std(roc_auc_scores)
    mean_time = np.mean(computation_times)
    std_time = np.std(computation_times)
    
    # Store results
    results[name] = {
        "Mean ROC AUC": mean_roc_auc,
        "STD ROC AUC": std_roc_auc,
        "Mean Time": mean_time,
        "STD Time": std_time
    }
    
    print(f"Mean ROC AUC: {mean_roc_auc:.3f} ± {std_roc_auc:.3f}")
    print(f"Mean Computation Time: {mean_time:.3f} ± {std_time:.3f} seconds")
    print()


Classifier: LR
Mean ROC AUC: 0.855 ± 0.004
Mean Computation Time: 0.033 ± 0.004 seconds

Classifier: DT
Mean ROC AUC: 0.836 ± 0.005
Mean Computation Time: 0.039 ± 0.003 seconds

Classifier: RF
Mean ROC AUC: 0.893 ± 0.005
Mean Computation Time: 2.355 ± 0.043 seconds

Classifier: KNN
Mean ROC AUC: 0.809 ± 0.007
Mean Computation Time: 2.285 ± 0.076 seconds

Classifier: XGB
Mean ROC AUC: 0.925 ± 0.003
Mean Computation Time: 0.194 ± 0.005 seconds

Classifier: SVM
Mean ROC AUC: 0.864 ± 0.005
Mean Computation Time: 440.240 ± 69.664 seconds

Classifier: MLP


C:\Users\P70071469\AppData\Local\anaconda3\envs\SLR\lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\P70071469\AppData\Local\anaconda3\envs\SLR\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\P70071469\AppData\Local\anaconda3\envs\SLR\lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\P70071469\AppData\Local\anaconda3\envs\SLR\lib\site-packages\keras\src\layers\core\dense.py:87

Mean ROC AUC: 0.885 ± 0.004
Mean Computation Time: 66.510 ± 11.560 seconds



In [25]:
import config_cat_embedding
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
#from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from scikeras.wrappers import KerasClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from tqdm import tqdm_notebook as tqdm

from data_prep import bank_data_prep, adult_data_prep
from embedding_helper import create_network
import time
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone


data_path = config_cat_embedding.paths['data']
data_path_out = config_cat_embedding.paths['data_output']
bank_data = pd.read_csv(data_path+'adult.csv', sep=',')
df_bank, cat_cols = adult_data_prep(bank_data)

X = df_bank.iloc[:, :-1]
target = df_bank.y

X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.2, random_state=1500)

# label-hot encoding
import category_encoders as ce
label_encoder = ce.OrdinalEncoder(cols=cat_cols) 
X_train_ohe = label_encoder.fit_transform(X_train)
X_test_ohe = label_encoder.transform(X_test)  # Use transform() instead of fit_transform()

# Standard scaling
stc = StandardScaler()
X_train_scaled = stc.fit_transform(X_train_ohe)
X_test_scaled = stc.transform(X_test_ohe)


# Define the classifiers
seed = 42
number_of_features = X_train_scaled.shape[1]  # Number of features in the input data

models = [
    ('LR', LogisticRegression(solver='lbfgs', random_state=seed, max_iter=1000)),
    ('DT', DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=seed)),
    ('RF', RandomForestClassifier(n_estimators=200, max_depth=5, random_state=seed, min_samples_leaf=3)),
    ('KNN', KNeighborsClassifier(n_neighbors=3)),
    ('XGB', XGBClassifier(eval_metric='logloss')),
    ('SVM', SVC(gamma='scale', random_state=seed, probability=True)),
    ('MLP', KerasClassifier(build_fn=create_network, number_of_features=number_of_features, epochs=100, batch_size=100, verbose=0))
]



from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone

# Define number of runs for cross-validation
n_runs = 5
kf = StratifiedKFold(n_splits=n_runs, shuffle=True, random_state=seed)

# Initialize dictionary to store results
results = {}

# Loop over the models
for name, model in models:
    print(f"Classifier: {name}")
    
    # Lists to store metrics for each fold
    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []
    roc_auc_scores = []
    computation_times = []
    
    # Perform cross-validation
    for train_idx, test_idx in kf.split(X_train_scaled, y_train):
        # Clone the model to ensure fresh training for each fold
        clf = clone(model)
        
        X_train_fold, X_test_fold = X_train_scaled[train_idx], X_train_scaled[test_idx]
        y_train_fold, y_test_fold = y_train.iloc[train_idx], y_train.iloc[test_idx]
        
        start_time = time.time()  # Start time
        
        clf.fit(X_train_fold, y_train_fold)
        y_pred = clf.predict(X_test_fold)#X_test_scaled
        y_pred_prob = clf.predict_proba(X_test_fold)[:, 1]  # Probabilities for ROC AUC
        
        end_time = time.time()  # End time
        computation_time = end_time - start_time  # Computation time
        
        # Append metrics for this fold
        accuracy_scores.append(accuracy_score(y_test_fold, y_pred))
        precision_scores.append(precision_score(y_test_fold, y_pred))
        recall_scores.append(recall_score(y_test_fold, y_pred))
        f1_scores.append(f1_score(y_test_fold, y_pred))
        roc_auc_scores.append(roc_auc_score(y_test_fold, y_pred_prob))
        computation_times.append(computation_time)
    
    # Compute mean and standard deviation of all metrics
    mean_accuracy = np.mean(accuracy_scores)
    std_accuracy = np.std(accuracy_scores)
    
    mean_precision = np.mean(precision_scores)
    std_precision = np.std(precision_scores)
    
    mean_recall = np.mean(recall_scores)
    std_recall = np.std(recall_scores)
    
    mean_f1 = np.mean(f1_scores)
    std_f1 = np.std(f1_scores)
    
    mean_roc_auc = np.mean(roc_auc_scores)
    std_roc_auc = np.std(roc_auc_scores)
    
    mean_time = np.mean(computation_times)
    std_time = np.std(computation_times)
    
    # Store results for the current model
    results[name] = {
        "Mean Accuracy": mean_accuracy, "STD Accuracy": std_accuracy,
        "Mean Precision": mean_precision, "STD Precision": std_precision,
        "Mean Recall": mean_recall, "STD Recall": std_recall,
        "Mean F1 Score": mean_f1, "STD F1 Score": std_f1,
        "Mean ROC AUC": mean_roc_auc, "STD ROC AUC": std_roc_auc,
        "Mean Time": mean_time, "STD Time": std_time
    }
    
    # Print results
    print(f"Mean Accuracy: {mean_accuracy:.3f} ± {std_accuracy:.3f}")
    print(f"Mean Precision: {mean_precision:.3f} ± {std_precision:.3f}")
    print(f"Mean Recall: {mean_recall:.3f} ± {std_recall:.3f}")
    print(f"Mean F1 Score: {mean_f1:.3f} ± {std_f1:.3f}")
    print(f"Mean ROC AUC: {mean_roc_auc:.3f} ± {std_roc_auc:.3f}")
    print(f"Mean Computation Time: {mean_time:.3f} ± {std_time:.3f} seconds")
    print()

# At the end, you can access the `results` dictionary for all the models


Classifier: LR
Mean Accuracy: 0.804 ± 0.004
Mean Precision: 0.688 ± 0.017
Mean Recall: 0.384 ± 0.013
Mean F1 Score: 0.493 ± 0.012
Mean ROC AUC: 0.855 ± 0.004
Mean Computation Time: 0.026 ± 0.001 seconds

Classifier: DT
Mean Accuracy: 0.815 ± 0.003
Mean Precision: 0.945 ± 0.013
Mean Recall: 0.270 ± 0.011
Mean F1 Score: 0.420 ± 0.014
Mean ROC AUC: 0.836 ± 0.005
Mean Computation Time: 0.038 ± 0.005 seconds

Classifier: RF
Mean Accuracy: 0.835 ± 0.005
Mean Precision: 0.868 ± 0.017
Mean Recall: 0.397 ± 0.020
Mean F1 Score: 0.544 ± 0.020
Mean ROC AUC: 0.893 ± 0.005
Mean Computation Time: 2.511 ± 0.333 seconds

Classifier: KNN
Mean Accuracy: 0.807 ± 0.005
Mean Precision: 0.621 ± 0.008
Mean Recall: 0.569 ± 0.018
Mean F1 Score: 0.593 ± 0.013
Mean ROC AUC: 0.809 ± 0.007
Mean Computation Time: 6.309 ± 1.639 seconds

Classifier: XGB
Mean Accuracy: 0.868 ± 0.006
Mean Precision: 0.776 ± 0.010
Mean Recall: 0.659 ± 0.018
Mean F1 Score: 0.713 ± 0.015
Mean ROC AUC: 0.925 ± 0.003
Mean Computation Time: 0

C:\Users\P70071469\AppData\Local\anaconda3\envs\SLR\lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\P70071469\AppData\Local\anaconda3\envs\SLR\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\P70071469\AppData\Local\anaconda3\envs\SLR\lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\P70071469\AppData\Local\anaconda3\envs\SLR\lib\site-packages\keras\src\layers\core\dense.py:87

Mean Accuracy: 0.830 ± 0.003
Mean Precision: 0.685 ± 0.022
Mean Recall: 0.592 ± 0.063
Mean F1 Score: 0.632 ± 0.027
Mean ROC AUC: 0.885 ± 0.004
Mean Computation Time: 69.639 ± 8.313 seconds

